This notebook uses an R kernel.

# Differential Gene Expression using DESeq2

Author: Zhongyi (James) Guo <br>
Date: 11/14/2024

## Import Packages

In [1]:
getwd()

[1] "/home/ubuntu/SCA-DL-DGE/script/deseq2"

In [2]:
.libPaths()

[1] "/home/ubuntu/miniconda3/lib/R/library"

In [3]:
library(tidyverse)
library(DESeq2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: S4Vectors



Loading required package: stats4



Loading required package: BiocGenerics



...

Loading required package: Biobase



Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.





Attaching package: ‘Biobase’




The following object is masked from ‘package:MatrixGenerics’:

    rowMedians




The following objects are masked from ‘package:matrixStats’:

    anyMissing, rowMedians




## Import Data

In [4]:
# count matrix
count_clean <- read_tsv('../../result/deseq2//count_clean.tsv') |> 
    as.data.frame() |>
    mutate(across(where(is.double), as.integer))
head(count_clean)

Rows: 58174 Columns: 9


── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): NAME
dbl (8): Sample_1, Sample_2, Sample_3, Sample_4, Sample_5, Sample_6, Sample_...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


,NAME,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSG00000223972,13,18,3,1,5,11,17,10
2,ENSG00000227232,1087,1002,182,531,200,114,319,172
3,ENSG00000278267,23,33,1,12,6,1,4,1
4,ENSG00000243485,0,3,0,1,2,2,0,2
5,ENSG00000284332,0,0,0,0,0,0,0,0
6,ENSG00000237613,0,0,0,0,0,0,2,0


In [5]:
# meta data
meta_data <- read_csv('../../result/deseq2/meta_data.csv')
meta_data$status <- as.factor(meta_data$status)
meta_data

Rows: 8 Columns: 2


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): sample, status



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sample,status
<chr>,<fct>
Sample_1,IS
Sample_2,IS
Sample_3,IS
Sample_4,IS
Sample_5,C
Sample_6,C
Sample_7,C
Sample_8,C


## DESeq2

In [6]:
dds <- DESeqDataSetFromMatrix(countData = count_clean, 
                              colData = meta_data, 
                              design = ~status, 
                              tidy = TRUE)

In [7]:
dds <- DESeq(dds)

estimating size factors



estimating dispersions



gene-wise dispersion estimates



mean-dispersion relationship



final dispersion estimates



fitting model and testing



In [8]:
res <- results(dds)

In [9]:
sig_gene <- res |>
    as.data.frame() |>
    filter(abs(log2FoldChange) > 2 & padj < 0.05) |>
    rownames_to_column('gene')
sig_gene

gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000278267,8.613466,2.615336,0.8545570,3.060458,2.209988e-03,7.991285e-03
ENSG00000268903,18.716412,5.774899,0.9753198,5.921032,3.199285e-09,6.290752e-08
ENSG00000269981,12.547480,5.752313,1.2084716,4.759990,1.936023e-06,1.891486e-05
ENSG00000239906,3.579557,3.791956,1.4671741,2.584531,9.751165e-03,2.777531e-02
ENSG00000279457,551.484304,2.532600,0.3827524,6.616811,3.670306e-11,1.079563e-09
ENSG00000250575,53.860707,5.050183,0.6465986,7.810384,5.701423e-15,3.577285e-13
ENSG00000225972,5794.202467,2.717834,0.5158510,5.268641,1.374375e-07,1.858197e-06
ENSG00000229344,1565.304483,2.235852,0.6209036,3.600965,3.170388e-04,1.543895e-03
ENSG00000230092,18.129602,-2.704409,0.6059129,-4.463362,8.068343e-06,6.512045e-05


In [10]:
dim(sig_gene)

[1] 4324    7

In [11]:
dim(sig_gene)[1] / dim(count_clean)[1]

[1] 0.07432874

14.27% of all genes were differentially expressed between IS and C.

## Save Data

In [12]:
saveRDS(dds, file = "../../result/deseq2/dds.rds")
write_csv(sig_gene, "../../result/deseq2/sig_gene.csv")

## Conclusion

In this notebook, we used DESeq2 to perform differential gene expression analysis, identifying genes with adjusted p-values below 0.05 and an absolute log2 fold change greater than 1, indicating a two-fold expression change between IS and Control conditions. These genes were statistically significant.

In [13]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/ubuntu/miniconda3/lib/libopenblasp-r0.3.28.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] DESeq2_1.42.1               SummarizedExperiment_1.32.0
 [3] Biobase_2.62.0              MatrixGenerics_1.14.0      
 [5] matrixStats_1.4.1           GenomicRanges_1.54.1       
 [7] GenomeInfoDb_1.38.8         IRanges_2.36.0             
 [9] S4Vectors_0.40.2            BiocGener